In [ ]:
import os
import sys
import time
import random
import string

import torch
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data
import numpy as np
import torch.nn.functional as F
from nltk.metrics.distance import edit_distance

from utils import CTCLabelConverter, CTCLabelConverterForBaiduWarpctc, AttnLabelConverter, Averager
from dataset import custom_dataset,AlignCollate
from model import Model

import easydict
global opt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

global opt
opt = easydict.EasyDict({
    "exp_name": "test_01",
    "train_data": "/data/data/STARN/data_lmdb_release/training",
    "valid_data":"/data/data/STARN/data_lmdb_release/validation",
    "manualSeed": 1111,
    "workers": 8,
    "batch_size":1024,
    "num_iter":300000,
    "valInterval":1,
    "saved_model":'',
    "FT":False,
    "adam":False,
    "lr":1,
    "beta1":0.9,
    "rho":0.95,
    "eps":1e-8,
    "grad_clip":5,
    "baiduCTC":False,
    "select_data":'ST',
    "batch_ratio":'1',
    "total_data_usage_ratio":'1.0',
    "batch_max_length":25,
    "imgW":100,
    "imgH":32,
    "rgb":False,
    "character":"0123456789abcdefghijklmnopqrstuvwxyz",
    "sensitive":False,
    "PAD":False,
    "data_filtering_off":False,
    "Transformation":"TPS",
    "FeatureExtraction":"ResNet",
    "SequenceModeling":"BiLSTM",
    "Prediction":'Attn',
    "num_fiducial":20,
    "input_channel":1,
    "output_channel":512,
    "hidden_size":256    
})





def validation(model, criterion, evaluation_loader, converter, opt):
    """ validation or evaluation """
    n_correct = 0
    norm_ED = 0
    length_of_data = 0
    infer_time = 0
    valid_loss_avg = Averager()

    for i, (image_tensors, labels) in enumerate(evaluation_loader):
        batch_size = image_tensors.size(0)
        length_of_data = length_of_data + batch_size
        image = image_tensors.to(device)
        # For max length prediction
        length_for_pred = torch.IntTensor([opt.batch_max_length] * batch_size).to(device)
        text_for_pred = torch.LongTensor(batch_size, opt.batch_max_length + 1).fill_(0).to(device)

        text_for_loss, length_for_loss = converter.encode(labels, batch_max_length=opt.batch_max_length)

        start_time = time.time()
        if 'CTC' in opt.Prediction:
            preds = model(image, text_for_pred)
            forward_time = time.time() - start_time

            # Calculate evaluation loss for CTC deocder.
            preds_size = torch.IntTensor([preds.size(1)] * batch_size)
            # permute 'preds' to use CTCloss format
            if opt.baiduCTC:
                cost = criterion(preds.permute(1, 0, 2), text_for_loss, preds_size, length_for_loss) / batch_size
            else:
                cost = criterion(preds.log_softmax(2).permute(1, 0, 2), text_for_loss, preds_size, length_for_loss)

            # Select max probabilty (greedy decoding) then decode index to character
            if opt.baiduCTC:
                _, preds_index = preds.max(2)
                preds_index = preds_index.view(-1)
            else:
                _, preds_index = preds.max(2)
            preds_str = converter.decode(preds_index.data, preds_size.data)
        
        else:
            preds = model(image, text_for_pred, is_train=False)
            forward_time = time.time() - start_time

            preds = preds[:, :text_for_loss.shape[1] - 1, :]
            target = text_for_loss[:, 1:]  # without [GO] Symbol
            cost = criterion(preds.contiguous().view(-1, preds.shape[-1]), target.contiguous().view(-1))

            # select max probabilty (greedy decoding) then decode index to character
            _, preds_index = preds.max(2)
            preds_str = converter.decode(preds_index, length_for_pred)
            labels = converter.decode(text_for_loss[:, 1:], length_for_loss)

        infer_time += forward_time
        valid_loss_avg.add(cost)

        # calculate accuracy & confidence score
        preds_prob = F.softmax(preds, dim=2)
        preds_max_prob, _ = preds_prob.max(dim=2)
        confidence_score_list = []
        for gt, pred, pred_max_prob in zip(labels, preds_str, preds_max_prob):
            if 'Attn' in opt.Prediction:
                gt = gt[:gt.find('[s]')]
                pred_EOS = pred.find('[s]')
                pred = pred[:pred_EOS]  # prune after "end of sentence" token ([s])
                pred_max_prob = pred_max_prob[:pred_EOS]


            if pred == gt:
                n_correct += 1

            '''
            (old version) ICDAR2017 DOST Normalized Edit Distance https://rrc.cvc.uab.es/?ch=7&com=tasks
            "For each word we calculate the normalized edit distance to the length of the ground truth transcription."
            if len(gt) == 0:
                norm_ED += 1
            else:
                norm_ED += edit_distance(pred, gt) / len(gt)
            '''

            # ICDAR2019 Normalized Edit Distance
            if len(gt) == 0 or len(pred) == 0:
                norm_ED += 0
            elif len(gt) > len(pred):
                norm_ED += 1 - edit_distance(pred, gt) / len(gt)
            else:
                norm_ED += 1 - edit_distance(pred, gt) / len(pred)

            # calculate confidence score (= multiply of pred_max_prob)
            try:
                confidence_score = pred_max_prob.cumprod(dim=0)[-1]
            except:
                confidence_score = 0  # for empty pred case, when prune after "end of sentence" token ([s])
            confidence_score_list.append(confidence_score)
            # print(pred, gt, pred==gt, confidence_score)

    accuracy = n_correct / float(length_of_data) * 100
    norm_ED = norm_ED / float(length_of_data)  # ICDAR2019 Normalized Edit Distance

    return valid_loss_avg.val(), accuracy, norm_ED, preds_str, confidence_score_list, labels, infer_time, length_of_data




if __name__ == '__main__':

    """ Seed and GPU setting """
    # print("Random Seed: ", opt.manualSeed)
    random.seed(opt.manualSeed)
    np.random.seed(opt.manualSeed)
    torch.manual_seed(opt.manualSeed)
    torch.cuda.manual_seed(opt.manualSeed)

    cudnn.benchmark = True
    cudnn.deterministic = True
    opt.num_gpu = torch.cuda.device_count()
    # print('device count', opt.num_gpu)
    if opt.num_gpu > 1:
        print('------ Use multi-GPU setting ------')
        print('if you stuck too long time with multi-GPU setting, try to set --workers 0')
        # check multi-GPU issue https://github.com/clovaai/deep-text-recognition-benchmark/issues/1
        opt.workers = opt.workers * opt.num_gpu
        opt.batch_size = opt.batch_size * opt.num_gpu









    numclass_path = "./ch_range.txt"
    f = open(numclass_path, 'r')
    ch_temp = f.read()
    f.close()
    
    
    opt.character = ch_temp

    converter = AttnLabelConverter(opt.character)
    opt.num_class = len(converter.character)


    train_dataset = custom_dataset("./dict/nia_refine_concat.txt","./font_full","train")
    valid_dataset = custom_dataset("./dict/nia_refine_concat.txt","./font","valid")

    AlignCollate_valid = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)

    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=opt.batch_size,
            shuffle=True,  # 'True' to check training progress with validation function.
            num_workers=int(opt.workers),
            collate_fn=AlignCollate_valid, pin_memory=True)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=opt.batch_size,
            shuffle=True,  # 'True' to check training progress with validation function.
            num_workers=int(opt.workers),
            collate_fn=AlignCollate_valid, pin_memory=True)


    if opt.rgb:
        opt.input_channel = 3
    model = Model(opt)


    for name, param in model.named_parameters():
        if 'localization_fc2' in name:
            print(f'Skip {name} as it is already initialized')
            continue
        try:
            if 'bias' in name:
                init.constant_(param, 0.0)
            elif 'weight' in name:
                init.kaiming_normal_(param)
        except Exception as e:  # for batchnorm.
            if 'weight' in name:
                param.data.fill_(1)
            continue


    model = torch.nn.DataParallel(model).to(device)



    criterion = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)

    loss_avg = Averager()


    filtered_parameters = []

    for p in filter(lambda p: p.requires_grad, model.parameters()):
        filtered_parameters.append(p)


    if opt.adam:
#         optimizer = optim.Adam(filtered_parameters, lr=opt.lr, betas=(opt.beta1, 0.999))
        optimizer = optim.Adam(filtered_parameters, lr=opt.lr)
    else:
        optimizer = optim.Adadelta(filtered_parameters, lr=opt.lr, rho=opt.rho, eps=opt.eps)

    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=360, gamma=0.98)
    
    nb_epochs = 100000

    for epoch in range(nb_epochs + 1):
        
        for batch_idx, samples in enumerate(train_loader):

            log = open(f'./log_val.txt', 'a')
            log2= open(f'./log_train.txt', 'a')

            start_time = time.time()        
            model.train()

            image_tensors, labels = samples
            image = image_tensors.to(device)
            text, length = converter.encode(labels, batch_max_length=opt.batch_max_length)




            preds = model(image, text[:, :-1])  # align with Attention.forward
            target = text[:, 1:]  # without [GO] Symbol
            cost = criterion(preds.view(-1, preds.shape[-1]), target.contiguous().view(-1))



            model.zero_grad()
            cost.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), opt.grad_clip)  # gradient clipping with 5 (Default)
            optimizer.step()
            scheduler.step()
            loss_avg.add(cost)

            for param_group in optimizer.param_groups:
                learning_rate_val=param_group['lr']


            ## 평가
            model.eval()
            with torch.no_grad():
                valid_loss, current_accuracy, current_norm_ED, preds, confidence_score, labels, infer_time, length_of_data = validation(
                            model, criterion, valid_loader, converter, opt)

            end = time.time()
            loss_log = f'epoch : {epoch} [{batch_idx}/{len(train_loader)}] Train loss: {loss_avg.val():0.5f},Valid loss: {valid_loss:0.5f}, time : {end-start_time} lr : {learning_rate_val}'        
            loss_avg.reset()


            print(loss_log)

            dashed_line = '-' * 80
            head = f'{"Ground Truth":25s} | {"Prediction":25s} | Confidence Score & T/F'
            predicted_result_log = f'{dashed_line}\n{head}\n{dashed_line}\n'
            for gt, pred, confidence in zip(labels[:5], preds[:5], confidence_score[:5]):
                if 'Attn' in opt.Prediction:
                    gt = gt[:gt.find('[s]')]
                    pred = pred[:pred.find('[s]')]

                predicted_result_log += f'{gt:25s} | {pred:25s} | {confidence:0.4f}\t{str(pred == gt)}\n'
            predicted_result_log += f'{dashed_line}'
    #         print(predicted_result_log)
            
            log2.write(loss_log + '\n')
            log.write(loss_log + '\n')
            log.write(predicted_result_log + '\n')
            log.close()
            log2.close()
            
#         scheduler.step()

------ Use multi-GPU setting ------
if you stuck too long time with multi-GPU setting, try to set --workers 0
Skip Transformation.LocalizationNetwork.localization_fc2.weight as it is already initialized
Skip Transformation.LocalizationNetwork.localization_fc2.bias as it is already initialized
epoch : 0 [0/21279] Train loss: 8.01477,Valid loss: 7.96785, time : 43.055087089538574 lr : 1
epoch : 0 [1/21279] Train loss: 7.90191,Valid loss: 7.95940, time : 10.63684606552124 lr : 1
epoch : 0 [2/21279] Train loss: 7.76160,Valid loss: 7.92126, time : 10.193422079086304 lr : 1
epoch : 0 [3/21279] Train loss: 7.57548,Valid loss: 7.62940, time : 10.5014328956604 lr : 1
epoch : 0 [4/21279] Train loss: 7.32230,Valid loss: 7.44721, time : 10.221692562103271 lr : 1
epoch : 0 [5/21279] Train loss: 7.04555,Valid loss: 6.98430, time : 10.762374639511108 lr : 1
epoch : 0 [6/21279] Train loss: 6.85669,Valid loss: 6.81894, time : 10.311545372009277 lr : 1
epoch : 0 [7/21279] Train loss: 6.76436,Valid loss:

epoch : 0 [84/21279] Train loss: 5.41221,Valid loss: 5.49292, time : 11.272991418838501 lr : 1
epoch : 0 [85/21279] Train loss: 5.40791,Valid loss: 5.43533, time : 11.626311779022217 lr : 1
epoch : 0 [86/21279] Train loss: 5.34941,Valid loss: 5.42027, time : 11.131417751312256 lr : 1
epoch : 0 [87/21279] Train loss: 5.35101,Valid loss: 5.40940, time : 11.41591501235962 lr : 1
epoch : 0 [88/21279] Train loss: 5.33876,Valid loss: 5.40374, time : 11.632770776748657 lr : 1
epoch : 0 [89/21279] Train loss: 5.34406,Valid loss: 5.39797, time : 12.163928270339966 lr : 1
epoch : 0 [90/21279] Train loss: 5.34067,Valid loss: 5.43872, time : 11.281046152114868 lr : 1
epoch : 0 [91/21279] Train loss: 5.39648,Valid loss: 5.46886, time : 11.134479999542236 lr : 1
epoch : 0 [92/21279] Train loss: 5.39831,Valid loss: 5.46828, time : 11.319393634796143 lr : 1
epoch : 0 [93/21279] Train loss: 5.38359,Valid loss: 5.43556, time : 13.137430667877197 lr : 1
epoch : 0 [94/21279] Train loss: 5.37351,Valid loss

epoch : 0 [170/21279] Train loss: 5.28152,Valid loss: 5.33538, time : 11.165393114089966 lr : 1
epoch : 0 [171/21279] Train loss: 5.28371,Valid loss: 5.35307, time : 11.623157501220703 lr : 1
epoch : 0 [172/21279] Train loss: 5.30113,Valid loss: 5.35610, time : 11.106384754180908 lr : 1
epoch : 0 [173/21279] Train loss: 5.35670,Valid loss: 5.36009, time : 11.06220817565918 lr : 1
epoch : 0 [174/21279] Train loss: 5.32081,Valid loss: 5.32951, time : 11.359291076660156 lr : 1
epoch : 0 [175/21279] Train loss: 5.27810,Valid loss: 5.34589, time : 13.612498998641968 lr : 1
epoch : 0 [176/21279] Train loss: 5.27699,Valid loss: 5.31777, time : 11.092890739440918 lr : 1
epoch : 0 [177/21279] Train loss: 5.27325,Valid loss: 5.36933, time : 10.929848194122314 lr : 1
epoch : 0 [178/21279] Train loss: 5.26845,Valid loss: 5.39074, time : 11.782737016677856 lr : 1
epoch : 0 [179/21279] Train loss: 5.30323,Valid loss: 5.41977, time : 11.316992998123169 lr : 1
epoch : 0 [180/21279] Train loss: 5.29604

epoch : 0 [256/21279] Train loss: 5.23680,Valid loss: 5.34691, time : 11.953730344772339 lr : 1
epoch : 0 [257/21279] Train loss: 5.23490,Valid loss: 5.34544, time : 14.809532880783081 lr : 1
epoch : 0 [258/21279] Train loss: 5.23347,Valid loss: 5.29346, time : 11.987007856369019 lr : 1
epoch : 0 [259/21279] Train loss: 5.24956,Valid loss: 5.29678, time : 11.74241328239441 lr : 1
epoch : 0 [260/21279] Train loss: 5.26004,Valid loss: 5.29438, time : 11.420980215072632 lr : 1
epoch : 0 [261/21279] Train loss: 5.27535,Valid loss: 5.27825, time : 11.69792890548706 lr : 1
epoch : 0 [262/21279] Train loss: 5.23707,Valid loss: 5.26438, time : 12.20431113243103 lr : 1
epoch : 0 [263/21279] Train loss: 5.22464,Valid loss: 5.29147, time : 11.829164743423462 lr : 1
epoch : 0 [264/21279] Train loss: 5.23387,Valid loss: 5.31997, time : 12.200077772140503 lr : 1
epoch : 0 [265/21279] Train loss: 5.29482,Valid loss: 5.31793, time : 10.906011819839478 lr : 1
epoch : 0 [266/21279] Train loss: 5.25193,V

epoch : 0 [342/21279] Train loss: 5.19425,Valid loss: 5.27383, time : 11.702513694763184 lr : 1
epoch : 0 [343/21279] Train loss: 5.19970,Valid loss: 5.25378, time : 11.35193419456482 lr : 1
epoch : 0 [344/21279] Train loss: 5.18005,Valid loss: 5.21135, time : 12.01891279220581 lr : 1
epoch : 0 [345/21279] Train loss: 5.17978,Valid loss: 5.29264, time : 11.769244909286499 lr : 1
epoch : 0 [346/21279] Train loss: 5.18000,Valid loss: 5.25759, time : 11.356447696685791 lr : 1
epoch : 0 [347/21279] Train loss: 5.22911,Valid loss: 5.27330, time : 12.079937219619751 lr : 1
epoch : 0 [348/21279] Train loss: 5.17796,Valid loss: 5.21624, time : 11.618089199066162 lr : 1
epoch : 0 [349/21279] Train loss: 5.16843,Valid loss: 5.26767, time : 11.781020641326904 lr : 1
epoch : 0 [350/21279] Train loss: 5.18131,Valid loss: 5.24291, time : 12.240309476852417 lr : 1
epoch : 0 [351/21279] Train loss: 5.24084,Valid loss: 5.28639, time : 13.618338584899902 lr : 1
epoch : 0 [352/21279] Train loss: 5.20673,

...

epoch : 0 [5962/21279] Train loss: 0.19101,Valid loss: 0.40601, time : 14.787677526473999 lr : 0.8953382542587163
epoch : 0 [5963/21279] Train loss: 0.19433,Valid loss: 0.24702, time : 13.20696759223938 lr : 0.8953382542587163
epoch : 0 [5964/21279] Train loss: 0.19933,Valid loss: 0.40147, time : 12.317876815795898 lr : 0.8953382542587163
epoch : 0 [5965/21279] Train loss: 0.19946,Valid loss: 0.39868, time : 12.730507612228394 lr : 0.8953382542587163
epoch : 0 [5966/21279] Train loss: 0.19549,Valid loss: 0.25638, time : 12.53849983215332 lr : 0.8953382542587163
epoch : 0 [5967/21279] Train loss: 0.19857,Valid loss: 0.26556, time : 12.189539670944214 lr : 0.8953382542587163
epoch : 0 [5968/21279] Train loss: 0.19151,Valid loss: 0.24858, time : 12.256743669509888 lr : 0.8953382542587163
epoch : 0 [5969/21279] Train loss: 0.18191,Valid loss: 0.24263, time : 12.298686265945435 lr : 0.8953382542587163
epoch : 0 [5970/21279] Train loss: 0.19150,Valid loss: 0.25890, time : 12.255898714065552 

epoch : 0 [6034/21279] Train loss: 0.18020,Valid loss: 0.25932, time : 12.759763956069946 lr : 0.8863848717161291
epoch : 0 [6035/21279] Train loss: 0.18265,Valid loss: 0.54856, time : 12.852579355239868 lr : 0.8863848717161291
epoch : 0 [6036/21279] Train loss: 0.21853,Valid loss: 0.41743, time : 12.51162075996399 lr : 0.8863848717161291
epoch : 0 [6037/21279] Train loss: 0.20316,Valid loss: 0.26839, time : 12.961869239807129 lr : 0.8863848717161291
epoch : 0 [6038/21279] Train loss: 0.20569,Valid loss: 0.25659, time : 12.830506086349487 lr : 0.8863848717161291
epoch : 0 [6039/21279] Train loss: 0.18651,Valid loss: 0.24955, time : 12.02243423461914 lr : 0.8863848717161291
epoch : 0 [6040/21279] Train loss: 0.19352,Valid loss: 0.24115, time : 12.268963098526001 lr : 0.8863848717161291
epoch : 0 [6041/21279] Train loss: 0.19244,Valid loss: 0.24254, time : 11.791554689407349 lr : 0.8863848717161291
epoch : 0 [6042/21279] Train loss: 0.18658,Valid loss: 0.25323, time : 12.103257656097412 

epoch : 0 [6107/21279] Train loss: 0.19950,Valid loss: 0.30430, time : 12.23923921585083 lr : 0.8863848717161291
epoch : 0 [6108/21279] Train loss: 0.20249,Valid loss: 0.29241, time : 12.592525005340576 lr : 0.8863848717161291
epoch : 0 [6109/21279] Train loss: 0.19237,Valid loss: 0.28250, time : 12.494659423828125 lr : 0.8863848717161291
epoch : 0 [6110/21279] Train loss: 0.19421,Valid loss: 0.30057, time : 13.240429639816284 lr : 0.8863848717161291
epoch : 0 [6111/21279] Train loss: 0.21260,Valid loss: 0.28675, time : 12.545653104782104 lr : 0.8863848717161291
epoch : 0 [6112/21279] Train loss: 0.19327,Valid loss: 0.29317, time : 12.99588942527771 lr : 0.8863848717161291
epoch : 0 [6113/21279] Train loss: 0.20656,Valid loss: 0.28381, time : 12.006412744522095 lr : 0.8863848717161291
epoch : 0 [6114/21279] Train loss: 0.19331,Valid loss: 0.27870, time : 14.705554485321045 lr : 0.8863848717161291
epoch : 0 [6115/21279] Train loss: 0.19237,Valid loss: 0.31022, time : 12.361741065979004 

epoch : 0 [6179/21279] Train loss: 0.17470,Valid loss: 0.26534, time : 12.637146234512329 lr : 0.8863848717161291
epoch : 0 [6180/21279] Train loss: 0.20235,Valid loss: 0.46128, time : 12.657594919204712 lr : 0.8863848717161291
epoch : 0 [6181/21279] Train loss: 0.19002,Valid loss: 0.24877, time : 12.825918912887573 lr : 0.8863848717161291
epoch : 0 [6182/21279] Train loss: 0.19870,Valid loss: 0.23773, time : 14.763952255249023 lr : 0.8863848717161291
epoch : 0 [6183/21279] Train loss: 0.19331,Valid loss: 0.23074, time : 12.682078838348389 lr : 0.8863848717161291
epoch : 0 [6184/21279] Train loss: 0.17929,Valid loss: 0.24034, time : 11.847204208374023 lr : 0.8863848717161291
epoch : 0 [6185/21279] Train loss: 0.18225,Valid loss: 0.24645, time : 12.433811902999878 lr : 0.8863848717161291
epoch : 0 [6186/21279] Train loss: 0.18125,Valid loss: 0.24995, time : 12.593262672424316 lr : 0.8863848717161291
epoch : 0 [6187/21279] Train loss: 0.18723,Valid loss: 0.23450, time : 12.80388832092285

epoch : 0 [6251/21279] Train loss: 0.19983,Valid loss: 0.23777, time : 12.071133375167847 lr : 0.8863848717161291
epoch : 0 [6252/21279] Train loss: 0.17838,Valid loss: 0.24696, time : 14.266223907470703 lr : 0.8863848717161291
epoch : 0 [6253/21279] Train loss: 0.17976,Valid loss: 0.23451, time : 12.356256008148193 lr : 0.8863848717161291
epoch : 0 [6254/21279] Train loss: 0.19147,Valid loss: 0.24594, time : 12.26571536064148 lr : 0.8863848717161291
epoch : 0 [6255/21279] Train loss: 0.18011,Valid loss: 0.24755, time : 11.820903301239014 lr : 0.8863848717161291
epoch : 0 [6256/21279] Train loss: 0.19618,Valid loss: 0.25778, time : 12.339102268218994 lr : 0.8863848717161291
epoch : 0 [6257/21279] Train loss: 0.16436,Valid loss: 0.23133, time : 12.536713123321533 lr : 0.8863848717161291
epoch : 0 [6258/21279] Train loss: 0.18741,Valid loss: 0.23271, time : 12.745091438293457 lr : 0.8863848717161291
epoch : 0 [6259/21279] Train loss: 0.16688,Valid loss: 0.23915, time : 12.449487447738647